In [1]:
from azure.ai.ml import Input, MLClient, Output, command
from azure.identity import ClientSecretCredential

# const values for Azure connection
SUBSCRIPTION_ID = "0a94de80-6d3b-49f2-b3e9-ec5818862801"
RESOURCE_GROUP = "buas-y2"
WORKSPACE_NAME = "NLP3"
TENANT_ID = "0a33589b-0036-4fe8-a829-3ed0926af886"
CLIENT_ID = "a2230f31-0fda-428d-8c5c-ec79e91a49f5"
CLIENT_SECRET = "Y-q8Q~H63btsUkR7dnmHrUGw2W0gMWjs0MxLKa1C"


credential = ClientSecretCredential(TENANT_ID, CLIENT_ID, CLIENT_SECRET)

ml_client = MLClient(
    subscription_id=SUBSCRIPTION_ID,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WORKSPACE_NAME,
    credential=credential,
)


compute = "adsai1"

In [2]:
from azure.ai.ml.entities import KubernetesOnlineEndpoint, KubernetesCompute, KubernetesOnlineDeployment
import datetime

endpoint_name = "test-endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint with a kubernetes

endpoint = KubernetesOnlineEndpoint(name=endpoint_name, description="test-endpoint", compute=compute, auth_mode="key", tags={"Type": "First test endpoint"})


In [3]:
print(f"Creating endpoint {endpoint_name}")

Creating endpoint test-endpoint-06241307490589


In [54]:
import sys
import os
# Install the Docker
!{sys.executable} -m pip install docker


In [4]:
#Create an endpoint
ml_client.online_endpoints.begin_create_or_update(endpoint).result()


KubernetesOnlineEndpoint({'provisioning_state': 'Succeeded', 'scoring_uri': 'http://194.171.191.227:30397/api/v1/endpoint/test-endpoint-06241307490589/score', 'openapi_uri': 'http://194.171.191.227:30397/api/v1/endpoint/test-endpoint-06241307490589/swagger.json', 'name': 'test-endpoint-06241307490589', 'description': 'test-endpoint', 'tags': {'Type': 'First test endpoint'}, 'properties': {'createdBy': 'a2230f31-0fda-428d-8c5c-ec79e91a49f5', 'createdAt': '2024-06-24T11:07:51.238873+0000', 'lastModifiedAt': '2024-06-24T11:07:51.238873+0000', 'azureml.onlineendpointid': '/subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/resourcegroups/buas-y2/providers/microsoft.machinelearningservices/workspaces/nlp3/onlineendpoints/test-endpoint-06241307490589', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/0a94de80-6d3b-49f2-b3e9-ec5818862801/providers/Microsoft.MachineLearningServices/locations/westeurope/mfeOperationsStatus/oeidp:12fa2fcc-0a79-4635-bc8c-8148b17bfac6:fb299b9a

In [5]:
from azure.ai.ml.entities import Environment, CodeConfiguration, ResourceRequirementsSettings, ResourceSettings
import os

registered_model_name = "RoBERTa_model"
latest_model_version = "2"
registered_environment_name = "BlockD"
latest_environment_version = "9"

model = ml_client.models.get(name=registered_model_name, version=latest_model_version)
env = ml_client.environments.get(name=registered_environment_name, version=latest_environment_version)


print(os.getcwd())
print(os.listdir("D:\BUAS\Year 2\Block 2D\Block D\BlockC\Juraj"))

blue_deployment = KubernetesOnlineDeployment(
    name="blue-deployment-test",
    endpoint_name=endpoint_name,
    model = model,
    environment = env,
    code_configuration=CodeConfiguration(code="D:\BUAS\Year 2\Block 2D\Block D\BlockC\Juraj",scoring_script="scoring.py"),
    instance_count = 1,
    resources=ResourceRequirementsSettings(
        requests=ResourceSettings(
            cpu="6000m",
            memory="20Gi"
        ),
    ),
)


d:\BUAS\Year 2\Block 2D\Block-D-Personal\Programming\Azure\Endpoint_BUAS
['pipeline.ipynb', 'scoring.py']


In [6]:
ml_client.begin_create_or_update(blue_deployment).result()

Check: endpoint test-endpoint-06241307490589 exists
Uploading Juraj (0.12 MBs): 100%|##########| 115992/115992 [00:00<00:00, 1296214.80it/s]




.......................................................................................................................................

HttpResponseError: (None) KubernetesOperationError: Operation failed in Kubernetes cluster. Reason:UserScriptImportError Details:Errors Occurred in User Script when importing modules, please check the deployment logs for more details. Please see troubleshooting guide, available here: https://aka.ms/amlarc-tsg
Code: None
Message: KubernetesOperationError: Operation failed in Kubernetes cluster. Reason:UserScriptImportError Details:Errors Occurred in User Script when importing modules, please check the deployment logs for more details. Please see troubleshooting guide, available here: https://aka.ms/amlarc-tsg
Exception Details:	(None) KubernetesOperationError: Operation failed in Kubernetes cluster. Reason:UserScriptImportError Details:Errors Occurred in User Script when importing modules, please check the deployment logs for more details. Please see troubleshooting guide, available here: https://aka.ms/amlarc-tsg
	Code: None
	Message: KubernetesOperationError: Operation failed in Kuber

In [50]:
import urllib.request
import requests
import json
import os
import ssl
import pandas as pd
# CHANGE URL PORT TO 3092!!!!


def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script

# Load the dataset
file_path = "D:/BUAS/Year 2/Block 2D/Block-D-Personal/Programming/Azure/Endpoint_BUAS/preprocess_output.csv"
dataframe = pd.read_csv(file_path)

# Convert the dataframe to a dictionary and then to a JSON string
data = dataframe.to_dict(orient='records')

body = str.encode(json.dumps(data))

url = "http://194.171.191.227:3092/api/v1/endpoint/test-endpoint-06241218613927/score"
# Replace this with the primary/secondary key, AMLToken, or Microsoft Entra ID token for the endpoint
api_key = 'zZQu0wUo284aViykl8acDOg3QmuYapg2'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'blue-deployment-test' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

The request failed with status code: 502
Server: nginx/1.18.0 (Ubuntu)
Date: Mon, 24 Jun 2024 10:56:56 GMT
Content-Type: application/json
Content-Length: 92
Connection: close
x-ms-run-function-failed: True
x-ms-server-version: azmlinfsrv/1.2.2
x-request-id: d5040552-95fe-41a4-b019-0fdeb6bd8cc3
x-ms-request-id: d5040552-95fe-41a4-b019-0fdeb6bd8cc3
assigned-version: blue-deployment-test-test-endpoint-06241218613927
assignment-method: HTTPHeader
versions-list: [{"name":"blue-deployment-test-test-endpoint-06241218613927","trafficPercentile":0}]


{"message": "An unexpected error occurred in scoring script. Check the logs for more info."}
